In [269]:
from qiskit_nature.drivers.second_quantization import HDF5Driver
driver_reduced = HDF5Driver("./PSPCz_reduced.hdf5")
properties = driver_reduced.run()


In [270]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
es_problem = ElectronicStructureProblem(driver_reduced)
second_q_op = es_problem.second_q_ops()
print(len(second_q_op))

4


In [347]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper 
qubit_converter = QubitConverter(JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
print(qubit_op)

-0.45781773131305903 * IIII
- 0.009666607989543467 * ZIII
+ 0.12689900731767084 * IZII
+ 0.030293077447785 * ZZII
- 0.009666607989543479 * IIZI
+ 0.03732964036584735 * ZIZI
+ 0.034223590264106186 * IZZI
+ 0.12689900731767084 * IIIZ
+ 0.034223590264106186 * ZIIZ
+ 0.05698995686717464 * IZIZ
+ 0.030293077447785 * IIZZ
+ 0.00014809461815615455 * XXII
+ 0.00014809461815615455 * YYII
- 7.872869677230731e-05 * XXZI
- 7.872869677230731e-05 * YYZI
+ 6.938452207544002e-05 * XXIZ
+ 6.938452207544002e-05 * YYIZ
+ 0.00014809461815615455 * IIXX
- 7.872869677230731e-05 * ZIXX
+ 6.938452207544002e-05 * IZXX
+ 0.00014809461815615455 * IIYY
- 7.872869677230731e-05 * ZIYY
+ 6.938452207544002e-05 * IZYY
+ 0.003930512816321183 * XXXX
+ 0.003930512816321183 * YYXX
+ 0.003930512816321183 * XXYY
+ 0.003930512816321183 * YYYY


In [348]:
from qiskit_nature.circuit.library import HartreeFock
init_state = HartreeFock(4, (1,1), qubit_converter)
init_state.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────

In [349]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(num_qubits = 4, 
                      su2_gates = ['h', 'ry'],
                      entanglement = 'full',
                      reps = 2,
                      initial_state = init_state)
ansatz.decompose().draw()


┌───┐   ┌───┐    ┌──────────┐                    ┌───┐┌──────────┐»
q_0: ┤ X ├───┤ H ├────┤ Ry(θ[0]) ├──■────■─────────■──┤ H ├┤ Ry(θ[4]) ├»
     ├───┤┌──┴───┴───┐└──────────┘┌─┴─┐  │         │  └───┘└──┬───┬───┘»
q_1: ┤ H ├┤ Ry(θ[1]) ├────────────┤ X ├──┼────■────┼────■─────┤ H ├────»
     ├───┤└──┬───┬───┘┌──────────┐└───┘┌─┴─┐┌─┴─┐  │    │     └───┘    »
q_2: ┤ X ├───┤ H ├────┤ Ry(θ[2]) ├─────┤ X ├┤ X ├──┼────┼───────■──────»
     ├───┤┌──┴───┴───┐└──────────┘     └───┘└───┘┌─┴─┐┌─┴─┐   ┌─┴─┐    »
q_3: ┤ H ├┤ Ry(θ[3]) ├───────────────────────────┤ X ├┤ X ├───┤ X ├────»
     └───┘└──────────┘                           └───┘└───┘   └───┘    »
«                                            ┌───┐┌──────────┐            »
«q_0: ─────────────────■────────■─────────■──┤ H ├┤ Ry(θ[8]) ├────────────»
«     ┌──────────┐   ┌─┴─┐      │         │  └───┘└──┬───┬───┘┌──────────┐»
«q_1: ┤ Ry(θ[5]) ├───┤ X ├──────┼────■────┼────■─────┤ H ├────┤ Ry(θ[9]) ├»
«     └──┬───┬───┘┌──┴───┴───┐┌─┴─┐┌─┴─┐  │    │     └───┘    └──┬───┬───┘»
«q_2: ───┤ H ├────┤ Ry(θ[6]) ├┤ X ├┤ X ├──┼────┼───────■─────────┤ H ├────»
«        ├───┤    ├──────────┤└───┘└───┘┌─┴─┐┌─┴─┐   ┌─┴─┐       ├───┤    »
«q_3: ───┤ H ├────┤ Ry(θ[7]) ├──────────┤ X ├┤ X ├───┤ X ├───────┤ H ├────»
«        └───┘    └──────────┘          └───┘└───┘   └───┘       └───┘    »
«                  
«q_0: ─────────────
«                  
«q_1: ─────────────
«     ┌───────────┐
«q_2: ┤ Ry(θ[10]) ├
«     ├───────────┤
«q_3: ┤ Ry(θ[11]) ├
«     └───────────┘

In [350]:
from qiskit.providers.aer import StatevectorSimulator
from qiskit.algorithms.optimizers import COBYLA
backend = StatevectorSimulator()
optimizer = COBYLA()

In [351]:
import numpy as np
np.random.seed(5)
num_params = ansatz.num_parameters
initial_point = np.random.random(num_params)

In [352]:
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import GroundStateEigensolver
vqe = VQE(ansatz = ansatz,
         optimizer = optimizer,
         initial_point = initial_point,
         quantum_instance = backend)

vqe_ground_state_solver = GroundStateEigensolver(qubit_converter, vqe)
vqe_results = vqe_ground_state_solver.solve(es_problem)
print(vqe_results.eigenenergies[0])

(-0.7656650244391225+0j)


In [353]:
from qiskit_nature.algorithms import QEOM
qeom_excited_state_solver = QEOM(vqe_ground_state_solver, 'sd')
qeom_results = qeom_excited_state_solver.solve(es_problem)
print(qeom_results)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -4043.631987584531
  - computed part:      -0.765665024439
  - ActiveSpaceTransformer extracted energy part: -4042.866322560092
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -4043.426917298302
* Electronic excited state energy (Hartree): -4043.426917298302
> Total excited state energy (Hartree): -4043.426917298302
  2: 
* Electronic excited state energy (Hartree): -4043.395388322966
* Electronic excited state energy (Hartree): -4043.395388322966
> Total excited state energy (Hartree): -4043.395388322966
  3: 
* Electronic excited state energy (Hartree): -4043.085721824136
* Electronic excited state energy (Hartree): -4043.085721824136
> Total excited state energy (Hartree): -4043.085721824136
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000


In [354]:
bandgap = qeom_results.computed_energies[1] - qeom_results.computed_energies[0]
bandgap

0.20507028622962853

In [370]:
# for now we believe that qubit_op is all we need
from qiskit.opflow import CircuitStateFn, StateFn
from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki
from qiskit.circuit.parametervector import ParameterVector

H_op = qubit_op
ansatz = ansatz
ansatz.parameters

# problem solved
# ansatz has a parameterView
# but to pull out parameter vector we should get a parameter
# and then call it's vector parent which will be parameterVector
ry = ansatz.parameters[0].vector
ry
# print(ry[0].vector)
# print(ry)
# print()
# print(H_and_ansatz.parameters)

ParameterVector(name=θ, length=12)

In [371]:
H_meas = StateFn(H_op, is_measurement=True)
circuit_ansatz = CircuitStateFn(ansatz)
H_and_ansatz = H_meas @ circuit_ansatz

sampler = CircuitSampler(backend = backend)
params = np.random.random(num_params)
binded = H_and_ansatz.bind_parameters({ry:params})
sampled = sampler.convert(binded)
print(sampled.eval())

(-0.4998550517628653+0j)


In [372]:
from scipy.optimize import minimize
from qiskit import *
def cost(params):
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    sampler = CircuitSampler(backend = backend)
    binded = H_and_ansatz.bind_parameters({ry:params})
    sampled = sampler.convert(binded)
    return sampled.eval().real
params = np.random.random(num_params)
vqe_result = minimize(cost, params, method="Powell", tol=1e-3)

print('The vqe ground state energy using is: {}'.format(vqe_results.eigenenergies[0]))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))


The vqe ground state energy using is: (-0.7656650244391225+0j)
The estimated ground state energy from VQE algorithm is: -0.7647310899128605


In [373]:
optimal_params = vqe_result.x
optimal_params

array([-0.88978434, -0.01470771, -0.22845516,  0.29995122,  0.88033167,
       -0.02974509,  0.10884307,  0.7103629 ,  1.20298301,  1.28331949,
        1.48613465,  0.93651326])

In [374]:
opt_circ = ansatz.bind_parameters({ry:optimal_params})
zero_proj = (1 / 2**4 * (I + Z) ^ (I + Z) ^ (I + Z) ^ (I + Z))

zero_proj_meas = StateFn(zero_proj, is_measurement = True)
ansatz_and_opt_circ = ansatz.compose(opt_circ.inverse())
#Penalty_ansatz = CircuitStateFn(ansatz_and_opt_circ)
Penalty_ansatz = CircuitStateFn(ansatz + opt_circ.inverse())
Penalty_and_ansatz = zero_proj_meas @ Penalty_ansatz

H_tot = H_and_ansatz + Penalty_and_ansatz

def cost(params):
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    sampler = CircuitSampler(backend = backend)
    binded = H_tot.bind_parameters({ry:params})
    sampled = sampler.convert(binded)
    return sampled.eval().real
params = np.random.random(num_params)
vqd_result = minimize(cost, params, method="Powell", tol=1e-3)



In [375]:
print('The qeom excited state energy using is: {}'.format(qeom_results.eigenenergies[1]))
print('The estimated excited state energy from vqd algorithm is: {}'.format(vqd_result.fun))


The qeom excited state energy using is: (-0.560594738209494+0j)
The estimated excited state energy from vqd algorithm is: -0.6916669936530623


In [369]:
print(H_tot)

SummedOp([
  ComposedOp([
    OperatorMeasurement(-0.45781773131305903 * IIII
    - 0.009666607989543467 * ZIII
    + 0.12689900731767084 * IZII
    + 0.030293077447785 * ZZII
    - 0.009666607989543479 * IIZI
    + 0.03732964036584735 * ZIZI
    + 0.034223590264106186 * IZZI
    + 0.12689900731767084 * IIIZ
    + 0.034223590264106186 * ZIIZ
    + 0.05698995686717464 * IZIZ
    + 0.030293077447785 * IIZZ
    + 0.00014809461815615455 * XXII
    + 0.00014809461815615455 * YYII
    - 7.872869677230731e-05 * XXZI
    - 7.872869677230731e-05 * YYZI
    + 6.938452207544002e-05 * XXIZ
    + 6.938452207544002e-05 * YYIZ
    + 0.00014809461815615455 * IIXX
    - 7.872869677230731e-05 * ZIXX
    + 6.938452207544002e-05 * IZXX
    + 0.00014809461815615455 * IIYY
    - 7.872869677230731e-05 * ZIYY
    + 6.938452207544002e-05 * IZYY
    + 0.003930512816321183 * XXXX
    + 0.003930512816321183 * YYXX
    + 0.003930512816321183 * XXYY
    + 0.003930512816321183 * YYYY),
    CircuitStateFn(
         »

In [208]:
for x in H_tot.parameters:
    break
x.vector

ParameterVector(name=θ, length=12)

In [ ]:
# |a * <pk | pi>|^2 = a^2 * <pk|pi><pi|pk> = 
# a^2 * <pk| R|0><0|R* |pk> = a^2 * <pk|P|pk>
# <pk | R |0><0| R* |pk>
